In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from IPython.display import clear_output
import time
import catboost
import re
import optuna
import sys
sys.path.append('../../')
import m4
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer,
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin

col_names = []
with open('../data/Faults27x7_var','r') as f:
    for line in f:
        col_names.append(line.strip())
        
train_org = pd.read_csv('../data/train.csv')
test_org = pd.read_csv('../data/test.csv')
org_data = pd.read_csv('../data/Faults.NNA', delimiter='\s', engine='python', names=col_names)

X = train_org.drop(['id','Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps','Other_Faults'], axis=1)
pastry = train_org['Pastry'].copy()
z_scratch = train_org['Z_Scratch'].copy()
k_scatch = train_org['K_Scatch'].copy()
stains = train_org['Stains'].copy()
dirtiness = train_org['Dirtiness'].copy()
bumps = train_org['Bumps'].copy()
other_faults = train_org['Other_Faults'].copy()

ys = [pastry, z_scratch, k_scatch, stains, dirtiness, bumps, other_faults]
y_names = ['pastry', 'z_scratch', 'k_scatch', 'stains', 'dirtiness', 'bumps', 'other_faults']

class KMeansTransformer(BaseEstimator,TransformerMixin):
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=self.n_clusters, n_init=10, random_state=0)
        
    def fit(self,X, y=None):
        self.kmeans.fit(X)
        return self
        
    def transform(self,X):
        labels = self.kmeans.predict(X)
        return np.c_[X, labels]

In [2]:
y_params_dict = {}
for y_nm in y_names:
    y_params_dict[y_nm] = None

In [3]:
y_scores = {}

In [4]:
for y,y_name in zip(ys,y_names):
    
    print(f'trial: {y_name}\n')
    
    def objective_xgb(trial):
    
        cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        params = dict(
            n_estimators = trial.suggest_int('n_estimators',100,500),
            max_depth = trial.suggest_int('max_depth',2,64),
            max_leaves = trial.suggest_int('max_leaves',2,128),
            learning_rate = trial.suggest_float('learning_rate',0.001,0.3),
            gamma = trial.suggest_float('gamma',0.001,10),
            min_child_weight = trial.suggest_float('min_child_weight', 0.01,10),
            subsample = trial.suggest_float('subsample', 0.33,0.85),
            colsample_bytree = trial.suggest_float('colsample_bytree',0.33,0.7),
            reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
            reg_lambda = trial.suggest_float('reg_lambda', 0.001,0.1)
        )
        
        xgbc = xgb.XGBClassifier(random_state= 0, objective='binary:logistic', verbose=0,**params)
        
        pipe = Pipeline(
            steps = [
                ('scaler', StandardScaler()),
                ('model', xgbc)
            ]
        )
        
        score = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
        return score
    
    study_lgb = optuna.create_study(direction='maximize')
    
    study_lgb.optimize(objective_xgb, n_trials=1000, n_jobs=-1, show_progress_bar=True)
    
    best_params = study_lgb.best_params
    y_params_dict[y_name] = best_params
    
    best_score = study_lgb.best_value
    y_scores[y_name] = best_score
    
    time.sleep(5)
    clear_output()

OSError: [Errno 22] Invalid argument: '09_Mar_20:02:50.json'

In [22]:
desc = 'vanilla xgbc with hpt'

In [23]:
m4.saver(y_params_dict,y_scores,desc)

In [5]:
y_params_dict

{'pastry': {'n_estimators': 480,
  'max_depth': 31,
  'max_leaves': 10,
  'learning_rate': 0.03367043946743182,
  'gamma': 3.8855613835149065,
  'min_child_weight': 3.984518493054492,
  'subsample': 0.6800878588269862,
  'colsample_bytree': 0.49391450660584085,
  'reg_alpha': 0.08809403572323495,
  'reg_lambda': 0.06605055800961854},
 'z_scratch': {'n_estimators': 474,
  'max_depth': 49,
  'max_leaves': 9,
  'learning_rate': 0.025150617026351282,
  'gamma': 5.480485141696638,
  'min_child_weight': 3.2787655492442784,
  'subsample': 0.7149455194983128,
  'colsample_bytree': 0.35171884661635155,
  'reg_alpha': 0.01709990699405062,
  'reg_lambda': 0.013858064737466924},
 'k_scatch': {'n_estimators': 469,
  'max_depth': 18,
  'max_leaves': 27,
  'learning_rate': 0.013019184341527188,
  'gamma': 2.832684619899278,
  'min_child_weight': 8.380738408509018,
  'subsample': 0.7359245187944895,
  'colsample_bytree': 0.359439885664309,
  'reg_alpha': 0.003920819688913427,
  'reg_lambda': 0.0029385

In [6]:
y_scores

{'pastry': 0.8719346610914931,
 'z_scratch': 0.9614636187840118,
 'k_scatch': 0.9861060189207164,
 'stains': 0.9928379881879204,
 'dirtiness': 0.8966676031071316,
 'bumps': 0.8113068596570112,
 'other_faults': 0.7103917057756547}

In [26]:
pastry_params = y_params_dict['pastry']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', xgb.XGBClassifier(random_state= 0, objective='binary:logistic',**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [27]:
z_scratch_params = y_params_dict['z_scratch']

z_scratch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', xgb.XGBClassifier(random_state= 0, objective='binary:logistic', **z_scratch_params))
    ]
)
z_scratch_m.fit(X,z_scratch)
z_scratch_pred = z_scratch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [28]:
k_scatch_params = y_params_dict['k_scatch']

k_scatch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', xgb.XGBClassifier(random_state= 0, objective='binary:logistic',**k_scatch_params))
    ]
)
k_scatch_m.fit(X,k_scatch)
k_scatch_pred = k_scatch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [29]:
stains_params = y_params_dict['stains']

stains_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', xgb.XGBClassifier(random_state= 0, objective='binary:logistic',**stains_params))
    ]
)
stains_m.fit(X,stains)
stains_pred = stains_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [30]:
dirtiness_params = y_params_dict['dirtiness']

dirtiness_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', xgb.XGBClassifier(random_state= 0, objective='binary:logistic',**dirtiness_params))
    ]
)
dirtiness_m.fit(X,dirtiness)
dirtiness_pred = dirtiness_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [31]:
bumps_params = y_params_dict['bumps']

bumps_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', xgb.XGBClassifier(random_state= 0, objective='binary:logistic',**bumps_params))
    ]
)
bumps_m.fit(X,bumps)
bumps_pred = bumps_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [32]:
other_faults_params = y_params_dict['other_faults']

other_faults_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', xgb.XGBClassifier(random_state= 0, objective='binary:logistic',**other_faults_params))
    ]
)
other_faults_m.fit(X,other_faults)
other_faults_pred = other_faults_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [33]:
sub = pd.DataFrame({'id':test_org['id'].copy(), 'Pastry':pastry_pred, 'Z_Scratch':z_scratch_pred, 'K_Scatch':k_scatch_pred,
                    'Stains':stains_pred, 'Dirtiness':dirtiness_pred, 'Bumps':bumps_pred, 'Other_Faults':other_faults_pred})

In [34]:
sub.head()

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.542258,0.001265,0.005876,0.007873,0.023696,0.183601,0.344444
1,19220,0.270092,0.025952,0.007724,0.007884,0.157546,0.170423,0.320102
2,19221,0.001973,0.050067,0.041113,0.008226,0.007689,0.281818,0.477841
3,19222,0.171895,0.002315,0.001768,0.008436,0.012893,0.421060,0.412308
4,19223,0.002853,0.002297,0.002023,0.008303,0.009574,0.595401,0.339369


In [35]:
ts = 0
for yn,score in y_scores.items():
    ts += score
    
print(ts/7)

0.8901012079319912


In [37]:
sub.to_csv('../submissions/m9.csv', index=False)